# Chapter 8. Distributed Processing

Distributed processing, which involves executing tasks across multiple machines to solve a certain problem.

## 8.1 Introduction to distributed computing

Handling this ever increasing amount of data presents substantial challenges. Large companies and organizations often build clusters of machines designed to store, process, and analyze large and complex datasets. Similar datasets are also produced in dataintensive fields such as environmental sciences and health care. These large-scale datasets have been recently called big data. The analysis techniques applied to big data usually involve a combination of machine learning, information retrieval, and visualization

### 8.1.1 An introduction to mapreduce

MapReduce is a programming model that allows you to express algorithms for efficient execution on a distributed system

The MapReduce framework was used in cooperation with a distributed filesystem, the Google File System (GFS or GoogleFS), which was designed to partition and replicate data across the computing cluster. Partitioning was useful for storing and processing datasets that wouldn't fit on a single node while replication ensured that the system was able to handle failures gracefully

The programming model exposed by MapReduce is actually quite simple. The idea is to express the computation as a combination of two, fairly generic, steps: Map and Reduce.

Map takes a collection of data as input and produces a transformation on this data. What is generally emitted by Map is a series of key value pairs that can be passed to a Reduce step. The Reduce step will aggregate items with the same key and apply a function to the collection to form a usually smaller collection of values.

The prototypical example of the MapReduce model is the implementation of a word count; the program takes a series of documents as input, and returns, for each word, the total number of occurrences in the document collection. The Map operation will produce a (key, value) entry where the first element is the word and the second element is 1 (that's because every word contributes 1 to the final count).We then perform the reduce operation to aggregate all the elements of the same key and produce the global count for each of the words

If we implement our algorithm using the Map and Reduce operation, the framework implementation will ensure that data production and aggregation is done efficiently, by limiting the communication between nodes through clever algorithms

Imagine that you have a cluster with two nodes, and a partition of the data (this is usually found locally in each node) is loaded in each node from disk and is ready for processing. A mapper process is created in each node and processes the data to produce the intermediate results

Next, it is necessary to send the data to the reducer for further processing. In order to do this, however, it is necessary that all the items that possess the same key are shipped to the same reducer. This operation is called shuffling and is the principal communication task in the MapReduce model:

Note that, before the data exchange happens, it is necessary to assign a subset of keys to each reducer; this step is called partitioning. Once a reducer receives its own partition of keys, it is free to process data and write the resulting output on disk

## 8.2 Dask

### 8.2.1 Directed acyclic graphs

We can use a familiar Pythonic API to build an execution plan, and the framework will automatically split the workflow into tasks that will be shipped and executed on multiple processes or computers

Dask expresses its variables and operations as a Directed Acyclic Graph (DAG) that can be represented through a simple Python dictionary

To briefly illustrate how this works, we will implement the sum of two numbers with Dask. We will define our computational graph by storing the values of our input variables in the dictionary. The a and b input variables will be given a value of 2:

In [ ]:
conda install dask

In [1]:
dsk = {
    "a" : 2,
    "b" : 2,
}
dsk

{'a': 2, 'b': 2}

Each variable represents a node in the DAG. The next step necessary to build our DAG is the execution of operations on the nodes we just defined. In Dask, a task can be defined by placing a tuple containing a Python function and its positional arguments in the dsk dictionary. To implement a sum, we can add a new node, named result, (the actual name is completely arbitrary) with a tuple containing the function we intend to execute, followed by its arguments. This is illustrated in the following code:

In [2]:
dsk = {
    "a" : 2,
    "b" : 2,
    "result": (lambda x, y: x + y, "a", "b")
}

In [3]:
from operator import add
dsk = {
    "a" : 2,
    "b" : 2,
    "result": (add, "a", "b")
}

It's important to note that the arguments we intend to pass to the function are the "a" and "b" strings, which refer to the a and b nodes in the graph. Note that we didn't use any Dask-specific functions to define the DAG; this is the first indication of how the framework is flexible and lean since all manipulations are performed on simple and familiar Python dictionaries.

The execution of tasks is performed by a scheduler, which is a function that takes a DAG and the task or tasks we'd like to perform and returns the computed value. The default Dask scheduler is the dask.get function, which can be used as follows

In [4]:
import dask
res = dask.get(dsk, "result")
print(res)

4


Defining graphs using a simple dictionary is useful to understand how Dask does its magic and for debugging purposes. Raw dictionaries can also be used to implement more complex algorithms not covered by the Dask API.

### 8.2.2 Dask arrays

One of the main use-cases of Dask is the automatic generation of parallel array operations, which greatly simplifies the handling of arrays that don't fit into memory. The strategy employed by Dask is to split the array into a number of subunits that, in Dask array terminology, are called chunks

da.from_array function will return a da.array object that will handle the splitting of the original array into subunits of the specified chunk size.

In [ ]:
import numpy as np
import dask.array as da

a = np.random.rand(30)
a_da = da.from_array(a, chunks=10)

In [ ]:
dict(a_da.dask)

In [ ]:
N = 10000
chunksize = 1000
x_data = np.random.uniform(-1, 1, N)
y_data = np.random.uniform(-1, 1, N)
x = da.from_array(x_data, chunks=chunksize)
y = da.from_array(y_data, chunks=chunksize)
hit_test = x ** 2 + y ** 2 < 1
hits = hit_test.sum()
pi = 4 * hits / N

In [ ]:
pi.compute() # Alternative: pi.compute(get=dask.get)
# Result:
# 3.1804000000000001

In [ ]:
pi.visualize()

### 8.2.3 Dash bag and dataframe

In [1]:
import dask.bag as dab
dab.from_sequence(range(100), npartitions=4)

dask.bag<from_sequence, npartitions=4>

In [2]:
collection = dab.from_sequence(["the cat sat on the mat", "the dog sat on the mat"], npartitions=2)

In [ ]:
binop = lambda total, x: total + x["count"]
combine = lambda a, b: a + b
(collection
.map(str.split)
.concat()
.map(lambda x: {"word": x, "count": 1})
.foldby(lambda x: x["word"], binop, 0, combine, 0)
.compute())

Just like da.array provides an API that closely mirrors NumPy features, Dask DataFrame can be used as a distributed version of pandas.DataFrame.

In [ ]:
collection = dab.from_sequence(["the cat sat on the mat", "the dog sat on the mat"], npartitions=2)
words = collection.map(str.split).concat()
df = words.to_dataframe(['words'])
df.head()

In [ ]:
df.words.value_counts().compute()

### 8.2.4 Dask distributed

The first iterations of the Dask project were designed to run on a single computer using a thread-based or a process-based scheduler. Recently, the implementation of a new distributed backend can be used to set up and run Dask graphs on a network of computers

In [ ]:
conda install distributed

In [ ]:
from dask.distributed import Client
client = Client()

#### Manual cluster setup

## 8.3 Using pyspark

Contrary to Hadoop MapReduce, Spark is designed to process data interactively and supports APIs for the Java, Scala, and Python programming languages. Given its different architecture, especially by the fact that Spark keep results in memory, Spark is generally much faster than Hadoop MapReduce

### 8.3.1 Setting up spark and pyspark

### 8.3.2 Spark architecture

### 8.3.3 Resilient distributed datasets

### 8.3.4 Spark dataframe

## 8.4 Scientific computing with mpi4pi

## 8.5 Summary